# Метаклассы, наследование, абстрактные классы

## Metaclass vs class vs instance

![](https://python.astrotech.io/_images/oop-metaclass-inheritance.png)

In [117]:
type(type)

type

## Instance vs Inheritance

![](https://drive.google.com/uc?export=view&id=1E71wN2w20hZRGfocaaeguwDUd_TJpi7G)

**Абстрактный класс**  
   + НАСЛЕДОВАНИЕ
   + НЕЛЬЗЯ создать экземпляр
   + хотя бы один абстрактный метод (@abstactmethod)
   + наследуется от ABC
   + нужен для описания интерфейса дочерних классов


**Метакласс**
   + СОЗДАНИЕ ЭКЗЕМПЛЯРА
   + вот тут нужно обязательно осознать разницу между **объектом класса** и **объектом экземпляра класса**
   + экземпляры - КЛАССЫ (объекты класса)
   + type - метакласс, по умолчанию участвует в создании всех классов в питоне, все классы в питоне являются экземплярами type (или его наследников)
   + пользовательский - наследуется от type
   + нужен для динамического создания классов и всяких хитрых дел связанных с созданием классов

In [118]:
# метакласс
class MyMeta(type):
    def __new__(cls, *args, **kwargs):
        print('Called MyMeta.__new__()')
        return super().__new__(cls, *args, **kwargs)

In [119]:
# объект класса
class MyClass(metaclass=MyMeta):
    def __init__(self):
        print('Called MyClass.__init__()')

Called MyMeta.__new__()


In [120]:
# объект класса
type(MyClass) # MyClass - экземпляр класса MyMeta

__main__.MyMeta

In [121]:
# объект экзмепляра класса
my_instance = MyClass()

Called MyClass.__init__()


In [122]:
# объект экзмепляра класса
type(my_instance) # my_instance - экземпляр класса MyClass

__main__.MyClass

In [123]:
# наследование
# объект класса
class MyChildClass(MyClass):
    pass

Called MyMeta.__new__()


In [124]:
# объект класса
type(MyChildClass) # MyChildClass - тоже экземпляр класса MyMeta

__main__.MyMeta

In [125]:
# родительские классы объекта класса
MyChildClass.__bases__ # MyChildClass - наследник/подкласс класса MyClass

(__main__.MyClass,)

In [126]:
# объект экземпляра класса
my_child_instance = MyChildClass()

Called MyClass.__init__()


In [127]:
# объект экземпляра класса
type(my_child_instance)

__main__.MyChildClass

## ДЗ-3, задача B

Вам нужно создать класс GoodInt, который ведет себя так же, как и обычный int, за искючением того, что при создании объекта из ﬂoat или строки с дробным числом не отрезает дробную часть (как int), а округляет число до ближайшего целого (подойдет встроенный round).
Пример использования:
```
    print(GoodInt(5.5))  # 6  
    print(GoodInt(5.6))  # 6  
    print(GoodInt(5.4))  # 5  
    print(GoodInt(5))  # 5  
    print(GoodInt(’5.5’))  # 6  
    print(GoodInt(5.6) + GoodInt(’5.6’))  # 12
```

Почему нельзя просто переписать init у потомка int?

In [131]:
class GoodInt(int):
    def __init__(self, x):
        print('Calling GoodInt.__init__()')
        x = round(float(x))
        int.__init__(x)
        
        
    def __new__(cls, *args, **kwargs):
        print(f'Calling GoodInt.__new__() with args {args}, kwargs {kwargs}')
        return super().__new__(cls, *args, **kwargs)
        
print(GoodInt(5.5))  # 6  

Calling GoodInt.__new__() with args (5.5,), kwargs {}
Calling GoodInt.__init__()
5


In [133]:
class GoodInt(int):
     def __new__(cls, x):
        x = round(float(x))
        print(f'Calling GoodInt.__new__() with arg {x}')
        return super().__new__(cls, x)
    
     def __init__(self, *args, **kwargs):
        print('Calling GoodInt.__init__()')
        super().__init__()
        
print(GoodInt(5.5))  # 6  

Calling GoodInt.__new__() with arg 6
Calling GoodInt.__init__()
6


In [142]:
gi = GoodInt(5.5)

Calling GoodInt.__new__() with arg 6
Calling GoodInt.__init__()


In [143]:
gi.d = 2

In [144]:
gi.d

2

In [145]:
a = 6

In [146]:
a.d = 10

AttributeError: 'int' object has no attribute 'd'

**\_\_new\_\_()** is intended mainly to allow subclasses of immutable types (like int, str, or tuple) to customize instance creation. It is also commonly overridden in custom metaclasses in order to customize class creation.

Как делаются неизменяемые типы данных в питоне?

In [134]:
class Immutable:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    # переопределением __setattr__
    def __setattr__(self, key, value):
        raise AttributeError("LOL nope.")

In [135]:
im = Immutable(2, 3)
print(im)

AttributeError: LOL nope.

In [136]:
# давайте выясним, что пошло не так
class Immutable:
    def __init__(self, a, b):
        print('Immutable.__init__() called')
        self.a = a
        self.b = b
        
    def __new__(cls, *args, **kwargs):
        print('Immutable.__new__() called')
        new_instance = super().__new__(cls)
        print('Immutable instance created')
        return new_instance

    # переопределением __setattr__
    def __setattr__(self, key, value):
        print(f'Calling Immutable.__setattr__ for key {key}, value {value}')
        raise AttributeError("LOL nope.")
        
im = Immutable(2, 3)
print(im)

Immutable.__new__() called
Immutable instance created
Immutable.__init__() called
Calling Immutable.__setattr__ for key a, value 2


AttributeError: LOL nope.

Чтобы сделать неизменяемый тип данных (с атрибутами) нам нужно как-то задать их до момента создания экземпляра неизменямого типа (до вызова new), как правило это делается с помощью наследования от изменяемого типа

In [137]:
class Mutable:
    def __init__(self, a, b):
        print("Mutable.__init__() called.")
        self.a = a
        self.b = b


class ActuallyImmutable(Mutable):
    def __new__(cls, a, b):
        # создаем сначала изменяемый экзмепляр, которому можно задать атрибуты 
        thing = Mutable(a, b)
        # меняем тип объекта
        thing.__class__ = cls
        return thing

    # переопределаем __init__, чтобы родительский не вызывался
    def __init__(self, *args, **kwargs):
        print("ActuallyImmutable.__init__() called.")

    def __setattr__(self, key, value):
        raise AttributeError("LOL nope srsly.")

In [138]:
im = ActuallyImmutable(2, 3)

Mutable.__init__() called.
ActuallyImmutable.__init__() called.


In [139]:
type(im)

__main__.ActuallyImmutable

In [140]:
print(im)
print(im.a)
print(im.b)

2
3


In [141]:
im.a = 23

AttributeError: LOL nope srsly.